In [3]:
import torch
from torch import nn
from torch.autograd import Variable

from data_loader import DataLoader
from model import UniSkip
from config import *
from datetime import datetime, timedelta

from tensorboardX import SummaryWriter
import nltk

from vocab import load_dictionary

In [4]:
lr = 3e-4
save_loc = "./saved_models/skip-best".format(lr, VOCAB_SIZE)
mod = UniSkip()
if USE_CUDA:
    mod.cuda(CUDA_DEVICE)
mod.load_state_dict(torch.load(save_loc))

encoder = mod.encoder

language = "english"
d = DataLoader(sentences=[''], word_dict=load_dictionary('./dataset/'+language+'/corpus.txt.pkl'))

Making reverse dictionary


In [5]:
import gather

from torch.nn import functional as F

def prepare_test(sentence):
    return " ".join(nltk.word_tokenize(sentence))

def get_vector(sentence):
    indices = d.convert_sentence_to_indices(sentence)
    output, _ = encoder(torch.stack([indices]))
    return output
    
def get_closest_sentence(sentence, target_author):
    path = gather.get_corpus_location(language, target_author)
    author_sentences = DataLoader(path)
    
    target_vector = get_vector(prepare_test(sentence))
    
    batch_size = 64
    max_sim = 0
    max_sent = []
    n_sent = len(author_sentences.sentences)
    
    for i in range(0, 5000, batch_size):
        batch = []
        for j in range(i, min(i + batch_size, n_sent)):
            sent = author_sentences.sentences[j]
            ind = d.convert_sentence_to_indices(sent)
            batch.append(ind)
        output, _ = encoder(torch.stack(batch))
        
        sim = F.cosine_similarity(output, target_vector)
        val, ind = sim.max(0)

        if val > max_sim:
            max_sim = val
            max_sent = batch[ind]
    return max_sent

[nltk_data] Downloading package punkt to /home/lucas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
test_sentence = "Hello there, I'm happy."
target_author = "shakespeare"

In [7]:
res = get_closest_sentence(test_sentence, target_author)
print(res)

STD: OK
Loading text file at /home/lucas/Projets/DLDIY/dataset/english/shakespeare/t8.shakespeare.txt.std
Making dictionary for these words
Using cached dictionary at /home/lucas/Projets/DLDIY/dataset/english/shakespeare/t8.shakespeare.txt.std.pkl
Making reverse dictionary


/usr/lib/python3.7/site-packages/torch/nn/functional.py:1090: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


tensor([  1, 273,   2,  10, 132,  43,   3,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0], device='cuda:0')


In [8]:
sent = [x for x in res if x != 0]
sent = d.convert_indices_to_sentences(sent)

from sacremoses import MosesDetokenizer
detokenizer = MosesDetokenizer()

print(detokenizer.detokenize(sent.split(" "), return_str=True))

UNK God, I think so.
